In [1]:
import numpy as np
import pandas as pd
import sklearn
import math 
import random
from pred_score import *

In [2]:
#Fixing seed to get reproducible results
random.seed(3)
np.random.seed(3)

# AE3

In [3]:
#Load preprocess data
AE3= np.array(pd.read_csv ('../data/processed_data/AE3.csv'))
y = np.array(AE3[:,-1],dtype=int)
AE3 = AE3[:,0:-1]
AE3.shape

(333, 32285)

 ### Baseline (with all genes)

In [4]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE3,y)
print(model.score_, model.recovery)

0.3508771929824561 0.6396396396396397


### Baseline (random)

In [5]:
AE3_shuffle =  np.array(pd.DataFrame(AE3).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE3_shuffle,y)
print(model.score_, model.recovery)

0.0 0.6396396396396397


### Optimized subset (MIM)

In [6]:
genes_AE3 = np.squeeze(pd.read_csv ('../data/processed_data/AE3genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/AE3genes_bestMIM.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_AE3 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
AE3 = AE3[:,ind_opt_genes]
print(AE3.shape)

subset = np.ones((len(gene_optimized),))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

(333, 4820)
4820


In [7]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE3,y)
print(model.score_, model.recovery)

0.5935828877005348 0.7177177177177178


In [8]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = AE3, y= y)
print(model.score_, model.recovery)

(101, 333)
0.7115384615384616 0.6336336336336337


In [4]:
def plot_cutoff(x:np.array, y:np.array, all_genes:np.array, best_genes:np.array):
    cut_off = np.linspace(0.1,1,10)
    
    precision, recovery = [], []
    #precision_random, recovery_random = [], []
    precision_base, recovery_base = [], []
    
    #With all genes
    subset = np.ones((x.shape[1],))
    subsets = subsampling_genes(subset, 101, 0.25)
    for t in cut_off:
        model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = t)
        result  = model.fit_predict(X = x, y= y)
        precision_base = precision_base.append(model.score_) 
        recovery_base = recovery_base.append(model.recovery)
    
    #With the optimized gene set
    ind_opt_genes = []
    for gene in gene_optimized:
        ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(all_genes == gene))))
        ind_opt_genes= list(ind_opt_genes.astype(int))

    #Only keep the optimized genes
    x = x[:,ind_opt_genes]
    
    subset = np.ones((len(gene_optimized),))
    subsets = subsampling_genes(subset, 101, 0.25)
    
    for t in cut_off:
        model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = t)
        result  = model.fit_predict(X = x, y= y)
        precision = precision.append(model.score_) 
        recovery = recovery.append(model.recovery)
    
    # create figure and axis objects with subplots()
    fig,ax = plt.subplots()
    # make a plot
    ax.plot(cut_off, precision, color="red")
    ax.plot(cut_off, precision_base, color="blue")
    #ax.plot(cut_off, precision, colot="yellow")
    # set x-axis label
    ax.set_xlabel("cut off")
    # set y-axis label
    ax.set_ylabel("precision")
    
    # twin object for two different y-axis on the sample plot
    ax2=ax.twinx()
    # make a plot with different y-axis using second axis object
    ax2.plot(cut_off, recovery, color="red")
    #ax2.plot(cut_off, precision_random, color="yellow")
    ax2.plot(cut_off, precision_base, colot="blue")
    ax2.set_ylabel("recovery")
    plt.show()
        

In [5]:
genes_AE3 = np.squeeze(pd.read_csv ('../data/processed_data/AE3genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/AE3genes_bestMIM.csv'))
plot_cutoff(x=AE3, y=y, all_genes= genes_AE3, best_genes=gene_optimized)

KeyboardInterrupt: 

# AE4

In [5]:
#Load preprocess data
AE4 = np.array(pd.read_csv ('../data/processed_data/AE4.csv'))
y = np.array(AE4[:,-1],dtype=int)
AE4 = AE4[:,0:-1]
AE4.shape

(1473, 32285)

 ### Baseline (with all genes)

In [6]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE4,y)
print(model.score_, model.recovery)

0.028664495114006514 0.9280380176510523


### Baseline (random)

In [ ]:
AE4_shuffle =  np.array(pd.DataFrame(AE4).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE4_shuffle,y)
print(model.score_, model.recovery)

### Optimized subset (ANOVA)

In [ ]:
genes_AE4 = np.squeeze(pd.read_csv ('../data/processed_data/AE4genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/AE4genes_bestANOVA.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_AE4 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
AE4 = AE4[:,ind_opt_genes]
print(AE4.shape)

subset = np.ones((len(gene_optimized),))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

In [ ]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE4,y)
print(model.score_, model.recovery)

In [ ]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = AE4, y= y)
print(model.score_, model.recovery)

# AE7

In [9]:
#Load preprocess data
AE7 = np.array(pd.read_csv ('../data/processed_data/AE7.csv'))
y = np.array(AE7[:,-1],dtype=int)
AE7 = AE7[:,0:-1]
AE7.shape

(1473, 32285)

 ### Baseline (with all genes)

In [ ]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE7,y)
print(model.score_, model.recovery)

### Baseline (random)

In [ ]:
AE7_shuffle =  np.array(pd.DataFrame(AE7).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE7_shuffle,y)
print(model.score_, model.recovery)

### Optimized subset (ANOVA)

In [ ]:
genes_AE7 = np.squeeze(pd.read_csv ('../data/processed_data/AE7genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/AE7genes_bestMIM.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_AE7 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
AE7 = AE7[:,ind_opt_genes]
print(AE7.shape)

subset = np.ones((len(gene_optimized),))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

In [ ]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(AE7,y)
print(model.score_, model.recovery)

In [ ]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = AE7, y= y)
print(model.score_, model.recovery)

# DO

In [9]:
#Load preprocess data
D0 = np.array(pd.read_csv ('../data/processed_data/D0.csv'))
y = np.array(D0[:,-1],dtype=int)
D0 = D0[:,0:-1]
D0.shape

(461, 10310)

 ### Baseline (with all genes)

In [11]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D0,y)
print(model.score_, model.recovery)

0.6686746987951807 0.720173535791757


### Baseline (random)

In [12]:
D0_shuffle =  np.array(pd.DataFrame(D0).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D0_shuffle,y)
print(model.score_, model.recovery)

0.0 0.720173535791757


### Optimized subset (MIM)

In [13]:
genes_D0 = np.squeeze(pd.read_csv ('../data/processed_data/D0genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/D0genes_bestMIM.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_D0 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
D0 = D0[:,ind_opt_genes]
print(D0.shape)

subset = np.ones((len(gene_optimized),))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

(461, 2480)
2480


In [14]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D0,y)
print(model.score_, model.recovery)

0.9444444444444444 0.8590021691973969


In [15]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = D0, y = y)
print(model.score_, model.recovery)

(101, 461)
0.9680851063829787 0.8156182212581344


# D6

In [16]:
#Load preprocess data
D6 = np.array(pd.read_csv ('../data/processed_data/D6.csv'))
y = np.array(D6[:,-1],dtype=int)
D6 = D6[:,0:-1]
D6.shape

(154, 10372)

 ### Baseline (with all genes)

In [17]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D6,y)
print(model.score_, model.recovery)

0.5961538461538461 0.6753246753246753


### Baseline (random)

In [18]:
D6_shuffle =  np.array(pd.DataFrame(D6).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D6_shuffle,y)
print(model.score_, model.recovery)

0.038461538461538464 0.6753246753246753


### Optimized subset (ANOVA)

In [19]:
genes_D6 = np.squeeze(pd.read_csv ('../data/processed_data/D6genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/D6genes_bestANOVA.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_D6 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
D6 = D6[:,ind_opt_genes]
print(D6.shape)

subset = np.ones((len(gene_optimized),))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

(154, 830)
830


In [20]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D6,y)
print(model.score_, model.recovery)

0.9552238805970149 0.8701298701298701


In [21]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = D6, y = y)
print(model.score_, model.recovery)

(101, 154)
0.953125 0.8311688311688312


# D15

In [22]:
#Load preprocess data
D15 = np.array(pd.read_csv ('../data/processed_data/D15.csv'))
y = np.array(D15[:,-1],dtype=int)
D15 = D15[:,0:-1]
D15.shape

(178, 9977)

 ### Baseline (with all genes)

In [23]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D15,y)
print(model.score_, model.recovery)

0.8714285714285714 0.7865168539325843


### Baseline (random)

In [24]:
D15_shuffle =  np.array(pd.DataFrame(D15).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D15_shuffle,y)
print(model.score_, model.recovery)

0.014285714285714285 0.7865168539325843


### Optimized subset (ANOVA)

In [25]:
genes_D15 = np.squeeze(pd.read_csv ('../data/processed_data/D15genes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/D15genes_bestANOVA.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind_opt_genes = np.append(ind_opt_genes, int(np.squeeze(np.where(genes_D15 == gene))))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
D15 = D15[:,ind_opt_genes]
print(D15.shape)

subset = np.ones((len(gene_optimized),))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

(178, 110)
110


In [26]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(D15,y)
print(model.score_, model.recovery)

1.0 0.9775280898876404


In [27]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = D15, y = y)
print(model.score_, model.recovery)

(101, 178)
1.0 0.9775280898876404


# LK

In [75]:
#Load preprocess data
LK = np.array(pd.read_csv ('../data/processed_data/LK.csv'))
y = np.array(LK[:,-1],dtype=int)
LK = LK[:,0:-1]
LK.shape

(610, 8405)

 ### Baseline (with all genes)

In [29]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LK,y)
print(model.score_, model.recovery)

0.20603015075376885 0.8131147540983606


### Baseline (random)

In [30]:
LK_shuffle =  np.array(pd.DataFrame(LK).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LK_shuffle,y)
print(model.score_, model.recovery)

0.0 0.8131147540983606


### Optimized subset (MIM)

In [3]:
genes_LK = np.squeeze(pd.read_csv ('../data/processed_data/LKgenes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/LKgenes_bestMIM.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind = np.squeeze(np.where(genes_LK == gene))
    if ind.shape == ():
        ind_opt_genes = np.append(ind_opt_genes, int(ind))
    else:
        for i in ind:
                ind_opt_genes = np.append(ind_opt_genes, int(i))
            
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
LK = LK[:,ind_opt_genes]
print(LK.shape)

subset = np.ones((LK.shape[1],))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

NameError: name 'LK' is not defined

In [77]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LK,y)
print(model.score_, model.recovery)

0.40173410404624277 0.7016393442622951


In [78]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = LK, y = y)
print(model.score_, model.recovery)

(101, 610)
0.6255924170616114 0.4737704918032787


### Prediction by barcode library

In [5]:
for i in range(0,9):
    
    print('-------------------------')
    print('Barcode library ' + str(i))
    LK = np.array(pd.read_csv ('../data/processed_data/LKlib' + str(i) + '.csv')).T
    y_LK  = np.squeeze(np.array(pd.read_csv('../data/processed_data/y_LK_lib' + str(i) + '.csv')))
    genes_LK = np.squeeze(pd.read_csv ('../data/processed_data/LKgenes_interest.csv'))
    gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/LKgenes_bestMIM.csv'))

    ind_opt_genes = []
    for gene in gene_optimized:
        ind = np.squeeze(np.where(genes_LK == gene))
        if ind.shape == ():
            ind_opt_genes = np.append(ind_opt_genes, int(ind))
        else:
            for i in ind:
                    ind_opt_genes = np.append(ind_opt_genes, int(i))

    ind_opt_genes= list(ind_opt_genes.astype(int))

    #Only keep the optimized genes
    LK = LK[:,ind_opt_genes]
    print(LK.shape)

    subset = np.ones((LK.shape[1],))
    subsets = subsampling_genes(subset, 101, 0.25)
    
    #Predict once
    model = FamiliesClusters(np.unique(y_LK),compute_precision,True)
    pred = model.fit_predict(LK,y_LK)
    print(model.score_, model.recovery)
    
    #Predict repetitively
    model = EnsemblingHierarchical(np.unique(y_LK),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
    result  = model.fit_predict(X = LK, y = y_LK)
    print(model.score_, model.recovery)

-------------------------
Barcode library 0
(105, 777)
0.05263157894736842 0.7238095238095238
0.1111111111111111 0.17142857142857143
-------------------------
Barcode library 1
(183, 777)
0.08695652173913043 0.7540983606557377
0.07142857142857142 0.15300546448087432
-------------------------
Barcode library 2
(244, 777)
0.010309278350515464 0.7950819672131147
0.1111111111111111 0.07377049180327869
-------------------------
Barcode library 3
(183, 777)
0.08695652173913043 0.7540983606557377
0.07692307692307693 0.14207650273224043
-------------------------
Barcode library 4
(2, 777)
0 1.0
0 1.0
-------------------------
Barcode library 5


EmptyDataError: No columns to parse from file

# LSK

In [87]:
#Load preprocess data
LSK = np.array(pd.read_csv ('../data/processed_data/LSK.csv'))
y = np.array(LSK[:,-1],dtype=int)
LSK = LSK[:,0:-1]
LSK.shape

(3383, 7570)

 ### Baseline (with all genes)

In [80]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LSK,y)
print(model.score_, model.recovery)

0.08357588357588358 0.9269878805793674


### Baseline (random)

In [81]:
LSK_shuffle =  np.array(pd.DataFrame(LSK).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LSK_shuffle,y)
print(model.score_, model.recovery)

0.0008316008316008316 0.9269878805793674


### Optimized subset (ANOVA)

In [88]:
genes_LSK = np.squeeze(pd.read_csv ('../data/processed_data/LSKgenes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/LSKgenes_bestANOVA.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind = np.squeeze(np.where(genes_LSK == gene))
    if ind.shape == ():
        ind_opt_genes = np.append(ind_opt_genes, int(ind))
    else:
        for i in ind:
                ind_opt_genes = np.append(ind_opt_genes, int(i))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
LSK = LSK[:,ind_opt_genes]
print(LSK.shape)

subset = np.ones((LSK.shape[1],))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

(3383, 418)
418


In [89]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LSK,y)
print(model.score_, model.recovery)

0.15286041189931351 0.8288501330180313


In [ ]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = LSK, y = y)
print(model.score_, model.recovery)

(101, 3383)
0.42971887550200805 0.23558971327224357


### Prediction by barcode library

In [8]:
for i in range(0,13):
    
    print('-------------------------')
    print('Barcode library ' + str(i))
    LK = np.array(pd.read_csv ('../data/processed_data/LSKlib' + str(i) + '.csv')).T
    y_LK  = np.squeeze(np.array(pd.read_csv('../data/processed_data/y_LSK_lib' + str(i) + '.csv')))
    genes_LK = np.squeeze(pd.read_csv ('../data/processed_data/LSKgenes_interest.csv'))
    gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/LSKgenes_bestANOVA.csv'))

    ind_opt_genes = []
    for gene in gene_optimized:
        ind = np.squeeze(np.where(genes_LK == gene))
        if ind.shape == ():
            ind_opt_genes = np.append(ind_opt_genes, int(ind))
        else:
            for i in ind:
                    ind_opt_genes = np.append(ind_opt_genes, int(i))

    ind_opt_genes= list(ind_opt_genes.astype(int))

    #Only keep the optimized genes
    LK = LK[:,ind_opt_genes]
    print(LK.shape)

    subset = np.ones((LK.shape[1],))
    subsets = subsampling_genes(subset, 101, 0.25)
    
    #Predict once
    model = FamiliesClusters(np.unique(y_LK),compute_precision,True)
    pred = model.fit_predict(LK,y_LK)
    print(model.score_, model.recovery)
    
    #Predict repetitively
    model = EnsemblingHierarchical(np.unique(y_LK),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
    result  = model.fit_predict(X = LK, y = y_LK)
    print(model.score_, model.recovery)

-------------------------
Barcode library 0
(258, 325)


/Users/dormann/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dormann/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: cannot convert float NaN to integer

# LK/LSK mix

In [98]:
#Load preprocess data
LSKmix = np.array(pd.read_csv ('../data/processed_data/LSKmix.csv'))
y = np.array(LSKmix[:,-1],dtype=int)
LSKmix = LSKmix[:,0:-1]
LSKmix.shape

(1023, 7230)

 ### Baseline (with all genes)

In [92]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LSKmix,y)
print(model.score_, model.recovery)

0.13649425287356323 0.8651026392961877


### Baseline (random)

In [93]:
LSKmix_shuffle =  np.array(pd.DataFrame(LSKmix).sample(frac = 1))
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LSKmix_shuffle,y)
print(model.score_, model.recovery)

0.0028735632183908046 0.8651026392961877


### Optimized subset (ANOVA)

In [99]:
genes_LSKmix = np.squeeze(pd.read_csv ('../data/processed_data/LSKmixgenes_interest.csv'))
gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/LSKmixgenes_bestANOVA.csv'))

ind_opt_genes = []
for gene in gene_optimized:
    ind = np.squeeze(np.where(genes_LSKmix == gene))
    if ind.shape == ():
        ind_opt_genes = np.append(ind_opt_genes, int(ind))
    else:
        for i in ind:
                ind_opt_genes = np.append(ind_opt_genes, int(i))
ind_opt_genes= list(ind_opt_genes.astype(int))

#Only keep the optimized genes
LSKmix = LSKmix[:,ind_opt_genes]
print(LSKmix.shape)

subset = np.ones((LSKmix.shape[1],))
subsets = subsampling_genes(subset, 101, 0.25)
print(len(subsets[0]))

(1023, 294)
294


In [100]:
model = FamiliesClusters(np.unique(y),compute_precision,True)
pred = model.fit_predict(LSKmix,y)
print(model.score_, model.recovery)

0.2084507042253521 0.8807429130009775


In [101]:
model = EnsemblingHierarchical(np.unique(y),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
result  = model.fit_predict(X = LSKmix, y = y)
print(model.score_, model.recovery)

(101, 1023)
0.4703196347031963 0.32942326490713586


### Prediction by barcode library

In [8]:
for i in range(0,6):
    
    print('-------------------------')
    print('Barcode library ' + str(i))
    LK = np.array(pd.read_csv ('../data/processed_data/LSKmixlib' + str(i) + '.csv')).T
    y_LK  = np.squeeze(np.array(pd.read_csv('../data/processed_data/y_LSKmix_lib' + str(i) + '.csv')))
    genes_LK = np.squeeze(pd.read_csv ('../data/processed_data/LSKmixgenes_interest.csv'))
    gene_optimized = np.squeeze(pd.read_csv ('../data/optimized_subsets/LSKmixgenes_bestANOVA.csv'))

    ind_opt_genes = []
    for gene in gene_optimized:
        ind = np.squeeze(np.where(genes_LK == gene))
        if ind.shape == ():
            ind_opt_genes = np.append(ind_opt_genes, int(ind))
        else:
            for i in ind:
                    ind_opt_genes = np.append(ind_opt_genes, int(i))

    ind_opt_genes= list(ind_opt_genes.astype(int))

    #Only keep the optimized genes
    LK = LK[:,ind_opt_genes]
    print(LK.shape)

    subset = np.ones((LK.shape[1],))
    subsets = subsampling_genes(subset, 101, 0.25)
    
    #Predict once
    model = FamiliesClusters(np.unique(y_LK),compute_precision,True)
    pred = model.fit_predict(LK,y_LK)
    print(model.score_, model.recovery)
    
    #Predict repetitively
    model = EnsemblingHierarchical(np.unique(y_LK),compute_precision,True,subsets = subsets, ensembling='voting', threshold_voting = 0.5)
    result  = model.fit_predict(X = LK, y = y_LK)
    print(model.score_, model.recovery)

-------------------------
Barcode library 0
(258, 325)


/Users/dormann/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/dormann/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: cannot convert float NaN to integer